## [200. Number of Islands](https://leetcode.com/problems/number-of-islands/)

針對每個是 '1' 的陸塊做dfs, 遍歷過的陸塊標示為 '*', 遇到新的陸塊結果+1

..

In [ ]:
class Solution:
    def dfs(self, grid, y, x):
        ## boundary
        ## not land
        if x < 0 or x >= len(grid[0]) or y < 0 or y >=len(grid) or grid[y][x] != '1':
            return
        ## visited
        grid[y][x] = '*'
        self.dfs(grid, y+1, x)
        self.dfs(grid, y, x+1)
        self.dfs(grid, y-1, x)
        self.dfs(grid, y, x-1)
    
    def numIslands(self, grid):
        res = 0
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == '1':
                    res += 1
                    self.dfs(grid, i, j)
        return res

## [1192. Critical Connections in a Network](https://leetcode.com/problems/critical-connections-in-a-network/)

回傳會造成 graph 被切成兩塊地連線

[Solution](https://leetcode.com/problems/critical-connections-in-a-network/discuss/382638/DFS-detailed-explanation-O(orEor)-solution)

* An edge is a critical connection, if and only if it is not in a cycle.

* rank: the depth of node during a DFS. The starting node has a rank 0. rank 會隨著 DFS 路徑增加.

* 如果找到比前一個點還小的 rank, 代表這個點已經在這次 DFS 中被遍歷過了, 只到 cycle!

* 如何讓其他 DFS level 知道這件事情: ```dfs``` 回傳最小 rank. 如果 dfs(v) 回傳值 <= rank(u), 那 v 一定是屬於 u (或其祖先的) cycle 之中, (u, v) 就不是答案了

1. 使用 defaultdict 建立 graph

2. 對所有點做 dfs, 去掉所有在 cycle 中的點. 剩下的 edge 就是 critical connection


...

In [ ]:
import collections
class Solution(object):
    def criticalConnections(self, n, connections):
        def makeGraph(connections):
            graph = collections.defaultdict(list)
            for conn in connections:
                graph[conn[0]].append(conn[1])
                graph[conn[1]].append(conn[0])
            return graph

        graph = makeGraph(connections)
        ## using set to delete duplicate edges and sort with begin node order
        connections = set(map(tuple, (map(sorted, connections))))
        ## -2 to inidcate non-visited edge
        rank = [-2] * n

        def dfs(node, depth):
            if rank[node] >= 0:
                # visiting (0<=rank<n), or visited (rank=n)
                return rank[node]
            rank[node] = depth
            min_back_depth = n
            for neighbor in graph[node]:
                if rank[neighbor] == depth - 1:
                    continue  # don't immmediately go back to parent. that's why i didn't choose -1 as the special value, in case depth==0.
                back_depth = dfs(neighbor, depth + 1)
                if back_depth <= depth:
                    connections.discard(tuple(sorted((node, neighbor))))
                min_back_depth = min(min_back_depth, back_depth)
            rank[node] = n  # this line is not necessary. see the "brain teaser" section below
            return min_back_depth
            
        dfs(0, 0)  # since this is a connected graph, we don't have to loop over all nodes.
        return list(connections)

In [ ]:
## faster
class Solution(object):
    def criticalConnections(self, n, connections):
        ## using 2D array for storing graph
        graph = [[] for i in range(n)]
        
        for n1, n2 in connections:
            graph[n1].append(n2)
            graph[n2].append(n1)

        lows = [n] * n ## the earlist timestamp to visit certain node
        critical = []  ## result
        
        def dfs(node, discovery_time, parent):
            if lows[node] == n: ## unvisited
                lows[node] = discovery_time
                for neighbor in graph[node]:
                    if neighbor != parent:      
                        expected_discovery_time_of_child = discovery_time + 1
                        actual_discovery_time_of_child = dfs(neighbor, expected_discovery_time_of_child, node)
                        ## the edge is not in the cycle: it's critical
                        if actual_discovery_time_of_child >= expected_discovery_time_of_child:
                            critical.append((node, neighbor))
                        lows[node] = min(lows[node], actual_discovery_time_of_child)
            return lows[node]
        
        dfs(0, 0, -1)
        return critical

## [394. Decode String](https://leetcode.com/problems/decode-string/)

iterative: go through the char

```repeator[token]```

```number```: update repeator

```[```: append token and repeator to the stack, reset token

```]```: pop the repeator and token, reset repeator

```char```: update token
...

In [ ]:
class Solution:
    def decodeString(self, s):
        stack = []
        token = ''
        repeator = 0
        for char in s:
            if char.isdigit():
                repeator = repeator * 10 + int(char)
            elif char == '[':
                stack.append(token)
                stack.append(repeator)
                token = ''
            elif char == ']':
                num = stack.pop()
                word = stack.pop()
                token = word + num * token
                repeator = 0
            else:
                token += char
        return token

## [547. Number of Provinces](https://leetcode.com/problems/number-of-provinces/)

return the number of group of the node

group of nodes: connected nodes

use a set to store all visited node

if encounter unvisited node, dfs its adjacent unvistied node

...

In [ ]:
## recursive

class Solution:
    def findCircleNum(self, M: List[List[int]]) -> int:
        visited = set()
        res = 0
        def dfs(idx):
            for node, neighbor in enumerate(M[idx]):
                if neighbor and node not in visited:
                    visited.add(node)
                    dfs(node)
        
        for node, neighbor in enumerate(M):
            if node not in visited:
                dfs(node)
                res += 1
        return res

## [199. Binary Tree Right Side View](https://leetcode.com/problems/binary-tree-right-side-view/)

### recursive DFS

time: O(N)

space: O(h), h is the height of the tree

...

In [ ]:
class Solution:
    def rightSideView(self, root: TreeNode) -> List[int]:
        if not root:
            return []
        
        rightside = []

        def dfs(node, level):
            ## must be the rightmost node
            if level == len(rightside):
                rightside.append(node.val)
            ## start from rightside
            for child in [node.right, node.left]:
                if child:
                    dfs(child, level+1)
        
        dfs(root, 0)
        return rightside

## [124. Binary Tree Maximum Path Sum](https://leetcode.com/problems/binary-tree-maximum-path-sum/)

可以產生的 max path, 有三種可能路經

```findMaxLeft```: node + Left

```findMaxRight```: node + Right

```triangle```: node + findMaxLeft + findMaxRight

time: O(N)

space: O(h), h is the height of the tree

...

In [ ]:
class Solution:
    def maxPathSum(self, root: TreeNode) -> int:
        res = -float('inf')
        def findMax(node):
            nonlocal res
            if not node:
                return 0
            findMaxLeft = max(0, findMax(node.left))
            findMaxRight = max(0, findMax(node.right))
            triangle = node.val + findMaxLeft + findMaxRight
            res = max(res, triangle)
            
            return node.val + max(findMaxLeft, findMaxRight)
        findMax(root)
        return res

## [695. Max Area of Island](https://leetcode.com/problems/max-area-of-island/)

Number of Islands 變體

記錄目前最大的面積, 遍歷過的標示成 0

...

In [ ]:
## recursive
class Solution:
    def maxAreaOfIsland(self, grid: List[List[int]]) -> int:
        def dfs(y, x):
            if x < 0 or x >= len(grid[0]) or y < 0 or y >= len(grid) or grid[y][x] != 1:
                return 0
            grid[y][x] = 0
            return (1 + 
                    dfs(y+1, x) + 
                    dfs(y-1, x) + 
                    dfs(y, x+1) + 
                    dfs(y, x-1)
                   )
        areas = []
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1:
                    areas.append(dfs(i,j))
        return max(areas) if areas else 0